In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, './..')
sys.path.insert(0, '../data')

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec
import proplot as pplt
from tqdm import tqdm

import numpy as np
import torch
from torchvision import datasets, transforms

from models import eval
from models import model as model_loader
import plots as pl
from utils import dev, load_data, classification

sys.path.insert(0, './../../')

import response_contour_analysis.utils.model_handling as model_utils
import response_contour_analysis.utils.dataset_generation as data_utils
import response_contour_analysis.utils.histogram_analysis as hist_utils
import response_contour_analysis.utils.principal_curvature as curve_utils
import response_contour_analysis.utils.plotting as plot_utils

# check device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

# Experiment parameters

In [ ]:
#experiment_params = dict()

#experiment_params['target_model_id'] = 0
#experiment_params['data_shape'] = images_nat[0,...].shape
#experiment_params['window_scale'] = 2.0
#experiment_params['num_edge_images'] = 30
#experiment_params['target'] = 0.0
#experiment_params['target_is_act'] = True

#experiment_params['num_images'] = int(experiment_params['num_edge_images']**2)
#experiment_params['x_range'] = (-experiment_params['window_scale'], experiment_params['window_scale'])
#experiment_params['y_range'] = experiment_params['x_range']
#experiment_params['device'] = DEVICE
#yx_range = experiment_params['yx_range'] = (experiment_params['y_range'], experiment_params['x_range'])

hess_params = dict()
hess_params['hessian_num_pts'] = 1e3#5e3
hess_params['hessian_lr'] = 1e-3
hess_params['hessian_random_walk'] = False
hess_params['return_points'] = False
num_iters = 2 # for paired image boundary search
num_steps_per_iter = 100 # for paired image boundary search
buffer_portion = 0.25
num_eps = 1000
batch_size = 1000
num_images = 5#50#labels_nat.size
hess_radius_mult = 0.5
num_advs = 4#8
autodiff = False
load = False

# Load data & models

In [ ]:
seed = 0

# load data
data_natural = np.load(f'../data/natural_{seed}.npy', allow_pickle=True).item()
advs_nat = data_natural['advs']
pert_lengths_nat = data_natural['pert_lengths']
classes_nat = data_natural['adv_class']
dirs_nat = data_natural['dirs']
images_nat = data_natural['images']
labels_nat = data_natural['labels']

data_madry = np.load(f'../data/robust_{seed}.npy', allow_pickle=True).item()
advs_madry = data_madry['advs']
pert_lengths_madry = data_madry['pert_lengths']
classes_madry = data_madry['adv_class']
dirs_madry = data_madry['dirs']
images_madry = data_madry['images']
labels_madry = data_madry['labels']

In [ ]:
mean_pert_lengths = np.mean([pert_lengths_nat[np.isfinite(pert_lengths_nat)].mean(),
                             pert_lengths_madry[np.isfinite(pert_lengths_madry)].mean()])
hess_params['hessian_dist'] = mean_pert_lengths * hess_radius_mult # radius around the target image

In [ ]:
# load models (all data assumes you are using the _diff model)
model_natural = model_loader.madry_diff()
model_madry = model_loader.madry_diff()
model_random = model_loader.madry_diff()
#if autodiff:
#    model_natural = model_loader.madry_diff()
#    model_madry = model_loader.madry_diff()
#    model_random = model_loader.madry_diff()
#else:
#    model_natural = model_loader.madry()
#    model_madry = model_loader.madry()
#    model_random = model_loader.madry()

model_natural.load_state_dict(torch.load(f'./../models/natural_{seed}.pt', map_location=DEVICE))
model_natural.to(DEVICE)

model_madry.load_state_dict(torch.load(f'./../models/robust_{seed}.pt', map_location=DEVICE))
model_madry.to(DEVICE)

model_random.load_state_dict(torch.load('./../models/random.pt', map_location=DEVICE))
model_random.to(DEVICE)

# Plane curvature for one pair of dirs

In [ ]:
def get_random_boundary_image(origin, eps=0.01, max_dist=4):
    num_channels, num_rows, num_cols = origin.shape
    direction = random_vector(num_channels * num_rows * num_cols)
    correct_label = torch.argmax(model(origin))
    pert_label = correct_label.clone()
    pert_image = origin.clone()
    num_steps = 0
    while pert_label == correct_label:
        pert_image = pert_image + direction * eps
        pert_label = torch.argmax(model(pert_image))
        num_steps += 1
    pert_image = origin + num_steps-1 * direction * eps
    small_eps = eps * 0.01
    num_small_steps = 0
    while pert_label == correct_label:
        pert_image = pert_image + direction * small_eps
        pert_label = torch.argmax(model(pert_image))
        num_small_steps += 1
    return pert_image
    

def random_vector(size):
    components = [np.random.normal() for i in range(size)]
    radius = np.sqrt(sum(x**2 for x in components))
    vect = np.array([x/radius for x in components])
    return vect


def get_adv_boundary_image(model, origin, direction, length, origin_class, adv_class, num_eps, buffer_portion, batch_size, autodiff):
    linspace_min = buffer_portion * length
    linspace_max = (1 + buffer_portion) * length
    eps_vals = np.linspace(linspace_min, linspace_max, num_eps)
    direction = direction.reshape(1, direction.size)
    eps_vals = eps_vals.reshape(-1, 1)
    origin = origin.reshape(1, origin.size)
    adv_line = origin + (direction * eps_vals)
    adv_line = adv_line.reshape(-1, 1, int(np.sqrt(origin.size)), int(np.sqrt(origin.size)))
    num_batches = int(np.ceil(num_eps / batch_size))
    input_batches = torch.split(torchify(adv_line), num_batches)
    model_outputs = np.empty((0, 10))
    for batch in input_batches:
        batch_outputs = model(batch).detach().cpu().numpy()
        model_outputs = np.concatenate((model_outputs, batch_outputs), axis=0)
    decision_scores = model_outputs[:, int(origin_class)] - model_outputs[:, int(adv_class)]
    boundary_index = np.abs(decision_scores).argmin()
    boundary_image = adv_line[boundary_index]
    return boundary_image

In [ ]:
def torchify(img):
    output = torch.from_numpy(img).type(torch.DoubleTensor).to(DEVICE) # always autodiff
    #if autodiff:
    #    output = torch.from_numpy(img).type(torch.DoubleTensor).to(DEVICE)
    #else:
    #    output = torch.from_numpy(img).type(torch.FloatTensor).to(DEVICE)
    return output


def get_paired_boundary_image(model, origin, alt_image, num_steps_per_iter, num_iters):
    num_channels, num_rows, num_cols = origin.shape
    input_shape = [1, num_channels, num_rows, num_cols]
    #correct_lbl = torch.argmax(model(torchify(origin.reshape(input_shape))))
    #final_lbl = torch.argmax(model(torchify(alt_image.reshape(input_shape))))
    #assert correct_lbl != final_lbl, (f'correct_lbl={correct_lbl}; final_lbl={final_lbl}')
    
    def find_pert(image_line):
        correct_lbl = torch.argmax(model(torchify(image_line[0, ...].reshape(input_shape))))
        pert_lbl = correct_lbl.clone()
        step_idx = 1 # already know the first one
        while pert_lbl == correct_lbl:
            pert_image = image_line[step_idx, ...]
            pert_lbl = torch.argmax(model(torchify(pert_image.reshape(input_shape))))
            step_idx += 1
        return step_idx-1, pert_image
    
    image_line = np.linspace(origin.reshape(-1), alt_image.reshape(-1), num_steps_per_iter)
    for search_iter in range(num_iters):
        step_idx, pert_image = find_pert(image_line)
        image_line = np.linspace(image_line[step_idx - 1, ...], image_line[step_idx, ...], num_steps_per_iter)
    delta_image = origin.reshape(-1) - pert_image
    pert_length = np.linalg.norm(delta_image)
    direction = delta_image / pert_length
    return pert_image.reshape(origin.shape), direction, pert_length


def generate_paired_dict(data_dict, model, num_images, num_advs):
    image_shape = data_dict['images'].shape[1:]
    num_pixels = int(np.prod(image_shape))
    images = np.zeros((num_images,) + image_shape)
    labels = np.zeros((num_images), dtype=np.int)
    dirs = np.zeros((num_images, num_advs, 1, num_pixels))
    advs = np.zeros((num_images, num_advs, 1, num_pixels))
    pert_lengths = np.zeros((num_images, num_advs))
    adv_class = np.zeros((num_images, num_advs))
    model_predictions = torch.argmax(model(torchify(data_dict['images'])), dim=1).detach().cpu().numpy()
    valid_indices = []
    for image_idx in range(data_dict['images'].shape[0]):
        if model_predictions[image_idx] == data_dict['labels'][image_idx]:
            valid_indices.append(image_idx)
    origin_indices = np.random.choice(valid_indices, size=num_images, replace=False)
    for image_idx, origin_idx in enumerate(origin_indices):
        images[image_idx, ...] = data_dict['images'][origin_idx, ...]
        labels[image_idx] = data_dict['labels'][origin_idx]
        shuffled_valid_indices = np.random.choice(valid_indices, size=len(valid_indices), replace=False)
        alt_indices = [idx for idx, alt_class in zip(shuffled_valid_indices, data_dict['labels'][shuffled_valid_indices]) if alt_class != labels[image_idx]]
        for dir_idx, alt_idx in enumerate(alt_indices[:num_advs]):
            alt_image = data_dict['images'][alt_idx, ...]
            boundary_image, boundary_dir, pert_length = get_paired_boundary_image(
                model, images[image_idx, ...], alt_image, num_steps_per_iter, num_iters)
            dirs[image_idx, dir_idx, ...] = boundary_dir.reshape(1, -1)
            advs[image_idx, dir_idx, ...] = boundary_image.reshape(1, -1)
            adv_class[image_idx,  dir_idx] = torch.argmax(model(torchify(boundary_image[None, ...]))).item()
            pert_lengths[image_idx, dir_idx] =  pert_length
    output_dict = {}
    output_dict['images'] = images
    output_dict['labels'] = labels
    output_dict['dirs'] = dirs
    output_dict['advs'] = advs
    output_dict['adv_class'] = adv_class
    output_dict['pert_lengths'] = pert_lengths
    return output_dict


def paired_activation(model, image, neuron1, neuron2):
    if not image.requires_grad:
        image.requires_grad = True
    model.zero_grad()
    activation1 = model_utils.unit_activation(model, image, neuron1, compute_grad=True)
    activation2 = model_utils.unit_activation(model, image, neuron2, compute_grad=True)
    activation_difference = activation1 - activation2
    return activation_difference


def paired_activation_and_gradient(model, image, neuron1, neuron2):
    activation_difference = paired_activation(model, image, neuron1, neuron2)
    grad = torch.autograd.grad(activation_difference, image)[0]
    return activation_difference, grad


def get_curvature(condition_zip, num_images, num_advs, num_eps, batch_size, buffer_portion, autodiff=False):
    """
    A note on the gradient of the difference in activations:
        The gradient points in the direction of the origin from the boundary image.
        Therefore, for large enough eps, origin - eps * grad/|grad| will reach the boundary; and boundary + eps * grad/|grad| will reach the origin 
    """
    models, model_data = zip(*condition_zip)
    num_models = len(models)
    image_shape = model_data[0]['images'][0,...][None,...].shape
    image_size = np.prod(image_shape)
    shape_operators = np.zeros((num_models, num_images, num_advs, image_size, image_size))
    principal_curvatures = np.zeros((num_models, num_images, num_advs, image_size))
    principal_directions = np.zeros((num_models, num_images, num_advs, image_size, image_size))
    mean_curvatures = np.zeros((num_models, num_images, num_advs))
    for model_idx, (model, data)  in enumerate(zip(models, model_data)):
        model_predictions = torch.argmax(model(torchify(data['images'])), dim=1).detach().cpu().numpy()
        valid_indices = [] # Need to ensure that all images are correctly labeled & have valid adversarial examples
        for image_idx in range(data['images'].shape[0]):
            if model_predictions[image_idx] == data['labels'][image_idx]: # correctly labeled
                if np.all(np.isfinite(data['pert_lengths'][image_idx, :num_advs])): # enough adversaries found
                    adv_predictions = data['advs']
                    valid_indices.append(image_idx)
        origin_indices = np.random.choice(valid_indices, size=num_images, replace=False)
        pbar = tqdm(total=num_images, leave=False)
        for image_idx, origin_idx in enumerate(origin_indices):
            clean_lbl = int(data['labels'][origin_idx])
            for adv_idx in range(num_advs):
                #adv_lbl = int(torch.argmax(model(torchify(data['advs'][origin_idx, adv_idx, ...].reshape(image_shape)))))
                #assert clean_lbl != adv_lbl, (f'get_curvature: clean_lbl={clean_lbl}, adv_lbl={adv_lbl}')
                boundary_image = get_paired_boundary_image(
                    model=model,
                    origin=data['images'][origin_idx, ...],
                    alt_image=data['advs'][origin_idx, adv_idx, ...],
                    num_steps_per_iter=num_steps_per_iter,
                    num_iters=num_iters
                )[0]
                adv_lbl = int(data['adv_class'][origin_idx, adv_idx])
                activation, gradient = paired_activation_and_gradient(model, torchify(boundary_image[None,...]), clean_lbl, adv_lbl)
                gradient = gradient.reshape(-1)
                if autodiff:
                    def func(x):
                        acts_diff = paired_activation(model, x, clean_lbl, adv_lbl)
                        return acts_diff
                    hessian = torch.autograd.functional.hessian(func, torchify(boundary_image[None,...]))
                    hessian = hessian.reshape((int(boundary_image.size), int(boundary_image.size)))
                else:
                    def func(x):
                        acts_diff, grad = paired_activation_and_gradient(model, x, clean_lbl, adv_lbl)
                        return acts_diff, grad
                    torch_image = torchify(boundary_image[None,...])
                    torch_image.requires_grad = True
                    hessian = curve_utils.sr1_hessian(
                        func, torch_image,
                        distance=hess_params['hessian_dist'],
                        n_points=hess_params['hessian_num_pts'],
                        random_walk=hess_params['hessian_random_walk'],
                        learning_rate=hess_params['hessian_lr'],
                        return_points=False,
                        progress=False)
                curvature = curve_utils.local_response_curvature(gradient, hessian)
                shape_operators[model_idx, image_idx, adv_idx, ...] = curvature[0].detach().cpu().numpy()
                principal_curvatures[model_idx, image_idx, adv_idx, :] = curvature[1].detach().cpu().numpy()
                principal_directions[model_idx, image_idx, adv_idx, ...] = curvature[2].detach().cpu().numpy()
                mean_curvatures[model_idx, image_idx, adv_idx] = np.mean(curvature[1].detach().cpu().numpy())
            pbar.update(1)
        pbar.close()
    return shape_operators, principal_curvatures, principal_directions, mean_curvatures

In [ ]:
def test_model_and_data(data_nat, data_mad):
    diff_str = '_diff' if autodiff else '_nodiff'
    for model, data, name in zip([model_natural, model_madry], [data_nat, data_mad], ['natural'+diff_str, 'madry'+diff_str]):
        image_shape = data['images'][0,...].shape
        clean_count = 0
        adv_count = 0
        clean_imgs = torchify(data['images'])
        clean_model_predictions = torch.argmax(model(clean_imgs), dim=1).detach().cpu().numpy()
        for img_idx in range(data['images'].shape[0]):
            label = int(data['labels'][img_idx])
            prediction = int(clean_model_predictions[img_idx])
            #assert  prediction == label , f'{name}: img_idx={img_idx}; prediction={prediction}; label={label}'
            #if clean_model_predictions[img_idx] != data['labels'][img_idx]: print(f'{name}: img_idx={img_idx}; prediction={clean_model_predictions[img_idx]}; label={data["labels"][img_idx]}')
            if clean_model_predictions[img_idx] != data['labels'][img_idx]: clean_count += 1
            adv_imgs = torchify(data['advs'][img_idx, ...].reshape((-1,)+image_shape))
            adv_model_predictions = torch.argmax(model(adv_imgs), dim=1).detach().cpu().numpy()
            for adv_idx in range(data['adv_class'].shape[1]):
                if np.isfinite(data['pert_lengths'][img_idx, adv_idx]):
                    label = int(data['adv_class'][img_idx, adv_idx])
                    prediction = int(adv_model_predictions[adv_idx])
                    #assert  prediction == label, f'{name}: img_idx={img_idx}, adv_idx={adv_idx}, prediction={prediction}, adv_label={label}'
                    #if adv_model_predictions[adv_idx] != data['adv_class'][img_idx, adv_idx]: print(f'{name}: img_idx={img_idx}, adv_idx={adv_idx}, prediction={adv_model_predictions[adv_idx]}, label={data["adv_class"][img_idx, adv_idx]}')
                    if adv_model_predictions[adv_idx] != data['adv_class'][img_idx, adv_idx]: adv_count += 1
        print(f'{name}: number of clean images with bad predictions = {clean_count}; number of adv images with bad predictions = {adv_count}')

#test_model_and_data(data_paired_natural, data_paired_madry)

In [ ]:
if autodiff:
    filename = '../data/curvatures_and_directions_autodiff.npz'
else:
    filename = '../data/curvatures_and_directions_sr1.npz'

if load:
    data = np.load(filename, allow_pickle=True)['data'].item()
    data_paired_natural = data['data_paired_natural']
    data_paired_madry = data['data_paired_madry']
    paired_shape_operators = data['paired_shape_operators']
    paired_principal_curvatures = data['paired_principal_curvatures']
    paired_principal_directions = data['paired_principal_directions']
    paired_mean_curvatures = data['paired_mean_curvatures']
    adv_shape_operators = data['adv_shape_operators']
    adv_principal_curvatures = data['adv_principal_curvatures']
    adv_principal_directions = data['adv_principal_directions']
    adv_mean_curvatures = data['adv_mean_curvatures']
else:
    data_paired_natural = generate_paired_dict(data_natural, model_natural, num_images, num_advs)
    data_paired_madry = generate_paired_dict(data_madry, model_madry, num_images, num_advs)
    paired_condition_zip = zip([model_natural, model_madry], [data_paired_natural, data_paired_madry])
    paired_shape_operators, paired_principal_curvatures, paired_principal_directions, paired_mean_curvatures = get_curvature(
        paired_condition_zip, num_images, num_advs, num_eps, batch_size, buffer_portion, autodiff)
    adv_condition_zip = zip([model_natural, model_madry], [data_natural, data_madry])
    adv_shape_operators, adv_principal_curvatures, adv_principal_directions, adv_mean_curvatures = get_curvature(
        adv_condition_zip, num_images, num_advs, num_eps, batch_size, buffer_portion, autodiff)
    save_dict = {
        'data_paired_natural':data_paired_natural,
        'dat_paired_madry':data_paired_madry,
        'paired_shape_operators': paired_shape_operators,
        'paired_principal_curvatures': paired_principal_curvatures,
        'paired_principal_directions': paired_principal_directions,
        'paired_mean_curvatures': paired_mean_curvatures,
        'adv_shape_operators': adv_shape_operators,
        'adv_principal_curvatures': adv_principal_curvatures,
        'adv_principal_directions': adv_principal_directions,
        'adv_mean_curvatures': adv_mean_curvatures
    }
    np.savez(filename, data=save_dict)

In [ ]:
colors = ['blue', 'orange']
bar_width = 0.5
fig, axs = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12,4))
fig.subplots_adjust(top=0.8)

for data_idx in range(2):
    mean_curvatures = [paired_mean_curvatures, adv_mean_curvatures][data_idx]
    for model_idx in range(2):
        boxprops = dict(color=colors[model_idx], linewidth=1.5, alpha=0.7)
        whiskerprops = dict(color=colors[model_idx], alpha=0.7)
        capprops = dict(color=colors[model_idx], alpha=0.7)
        medianprops = dict(linestyle='--', linewidth=0.5, color=colors[model_idx])
        meanpointprops = dict(marker='o', markeredgecolor='black',
                              markerfacecolor=colors[model_idx])
        meanprops = dict(linestyle='-', linewidth=0.5, color=colors[model_idx])
        data = mean_curvatures[model_idx, :, :].reshape(-1)
        axs[data_idx].boxplot(data, sym='', positions=[model_idx], whis=(5, 95), widths=bar_width, meanline=True, showmeans=True, boxprops=boxprops,
            whiskerprops=whiskerprops, capprops=capprops, medianprops=medianprops, meanprops=meanprops)
    axs[data_idx].set_xticks([0, 1], minor=False)
    axs[data_idx].set_xticks([], minor=True)
    axs[data_idx].set_xticklabels(['Naturally trained', 'Adversarially trained'])
    if data_idx == 0:
        axs[data_idx].set_ylabel('Mean curvature')
        axs[data_idx].set_title('Paired image boundary')
    else:
        axs[data_idx].set_title('Adversarial image boundary')

for model_idx in range(adv_mean_curvatures.shape[0]):
    boxprops = dict(color=colors[model_idx], linewidth=1.5, alpha=0.7)
    whiskerprops = dict(color=colors[model_idx], alpha=0.7)
    capprops = dict(color=colors[model_idx], alpha=0.7)
    medianprops = dict(linestyle='--', linewidth=0.5, color=colors[model_idx])
    meanpointprops = dict(marker='o', markeredgecolor='black',
                          markerfacecolor=colors[model_idx])
    meanprops = dict(linestyle='-', linewidth=0.5, color=colors[model_idx])
    for adv_idx in range(adv_mean_curvatures.shape[-1]):
        data = adv_mean_curvatures[model_idx, :, adv_idx].reshape(-1)
        axs[2].boxplot(data, sym='', positions=[adv_idx], whis=(5, 95), widths=bar_width, meanline=True, showmeans=True, boxprops=boxprops,
            whiskerprops=whiskerprops, capprops=capprops, medianprops=medianprops, meanprops=meanprops)
axs[2].set_title('Adversarial image boundary')
axs[2].set_xlabel('Dimension number')
axs[2].set_xticks([i for i in range(adv_mean_curvatures.shape[-1])], minor=False)
axs[2].set_xticks([], minor=True)
axs[2].set_xticklabels([str(i+1) for i in range(adv_mean_curvatures.shape[-1])])

def make_space_above(axes, topmargin=1):
    """ increase figure size to make topmargin (in inches) space for 
        titles, without changing the axes sizes"""
    fig = axes.flatten()[0].figure
    s = fig.subplotpars
    w, h = fig.get_size_inches()

    figh = h - (1-s.top)*h  + topmargin
    fig.subplots_adjust(bottom=s.bottom*h/figh, top=1-topmargin/figh)
    fig.set_figheight(figh)


make_space_above(axs, topmargin=0.5)  

fig.suptitle(f'Curvature at the decision boundary\nfor {num_images} images and the first {num_advs} adversarial directions', y=1.0)
plt.show()
fig.savefig('../data/mean_curvature_boxplots.png', transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
curvature_indices = [0, 10, 100, -100, -10, -1]
#curvature_indices = [-100]

matrix_shape = adv_principal_curvatures.shape[1:3]
(image_idx, adv_idx) = [np.random.randint(low=0, high=matrix_shape[i]) for i in range(len(matrix_shape))]
while not np.isfinite(data_natural['pert_lengths'][image_idx, adv_idx]):
    (image_idx, adv_idx) = [np.random.randint(low=0, high=matrix_shape[i]) for i in range(len(matrix_shape))]
origin = data_natural['images'][image_idx, ...]#.reshape(-1)

boundary_image = get_paired_boundary_image(
    model=model_natural,
    origin=origin,
    alt_image=data_natural['advs'][image_idx, adv_idx, ...],
    num_steps_per_iter=num_steps_per_iter,
    num_iters=num_iters
)[0]
boundary_dist = np.linalg.norm(boundary_image.reshape(-1) - origin.reshape(-1))

figsize = 8
fig, axs = plt.subplots(nrows=len(curvature_indices), ncols=1, figsize=(figsize, figsize*len(curvature_indices)))
for ax_idx, curvature_idx in enumerate(curvature_indices):
    ax = axs[ax_idx]
    random_index = (0, image_idx, adv_idx, curvature_idx, Ellipsis)
    adv1 = boundary_image.reshape(-1)
    adv2 = origin.reshape(-1) + boundary_dist * adv_principal_directions[random_index]
    model_ = model_natural
    pl.plot_dec_space(origin, adv1, adv2, model_, offset=1, n_grid=100, show_legend=True, show_advs=True, overlay_inbounds=True, ax=ax)
    ax.set_title(f'Curvature = {adv_principal_curvatures[random_index]:.5f}')
plt.show()

fig.savefig('../data/curvature_visualizations.png', transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
fig.savefig('../data/adv_fig.png', bbox_inches='tight')

In [ ]:
#activation, gradient = model_utils.unit_activation_and_gradient(model_natural, torchify(boundary_image[None,...]), clean_id)
#gradient = gradient.reshape(-1)
#abscissa = [gradient]
#ordinate = [adv_principal_directions[random_index]]